<figure>
  <IMG SRC="https://upload.wikimedia.org/wikipedia/commons/thumb/d/d5/Fachhochschule_Südwestfalen_20xx_logo.svg/320px-Fachhochschule_Südwestfalen_20xx_logo.svg.png" WIDTH=250 ALIGN="right">
</figure>

# Einführung Machine Learning
### Sommersemester 2022
Prof. Dr. Heiner Giefers

# Das Gradientenverfahren

In [2]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

In [3]:
# Test Case
m_c, n_c = 2, 4
np.random.seed(0)
X = np.random.randn(m_c, n_c)
theta_c = np.r_[[[0]],np.random.randn(n_c, 1)]
y_c = np.random.randn(m_c, 1)

Bei der Logistischen Regression haben wir das Gradientenverfahren benutzt, um die Parameter unseres Modells, einer linearen Funktion $$Z = f_{\theta}(x)=\theta_0+\theta_ix$$ transformiert durch die Aktivierungsfunktion $$\hat y = h_{\theta}(x) = \sigma(Z) = \frac{1}{1+e^{-Z}}$$ schrittweise zu verbessern.


Die Verbesserung, bzw. die Qualität des Modells, haben wir anhand der Kostenfunktion $$J_{\theta}(x)=-\frac{1}{m} \sum\limits_{i = 1}^{m} [y^{(i)}\log(\hat y^{(i)}) + (1-y^{(i)})\log(1- \hat y^{(i)})]$$ berechnet.

Wir wollen nun schrittweise die Modellfunktion, die Kostenfunktion sowie das Gradientenverfahren als Python-Funktionen definieren.
Um uns die Berechnungen zu vereinfachen, hängen wir an unseren Datensatz eine Spalte mit Einsen an.
Damit können wir alle Parameter $\theta$ (inklusive des Bias-Parameters) in einer Vektor-Operation verarbeiten.

In [4]:
X_c = np.c_[np.ones(X.shape[0]).T,X]
X_c

array([[ 1.        ,  1.76405235,  0.40015721,  0.97873798,  2.2408932 ],
       [ 1.        ,  1.86755799, -0.97727788,  0.95008842, -0.15135721]])

## Modellfunktion
**Aufgabe: Schreibe eine Funktion $f$, die folgende Parameter erhält:**
1. Die Matrix $X \in \mathbb{R}^{m\times{}n}$, die die Datenpunkte des Trainigsdatensatzes enthält.
2. Die Parameter $\theta$

**$f$ soll folgende lineare Funktion implementieren:** $$Z = f_{\theta}(X)=X\theta$$


In [10]:
def f(X, theta):
    """evaluates linear function.
    Arguments:
        X: value
        theta: Parameter
    """
    
    Z = np.matmul(X, theta)
    
    return Z

In [11]:
# Test Cell
#----------

Z = f(X_c, theta_c)
#----------
# f

assert Z.shape == (m_c, 1), 'Use correctly sequenced matrix multiplication'
assert np.isclose(Z[0], 3.38207), 'Expected 3.38207 but got %.5f' %Z[0]

del Z

**Aufgabe: Implementieren Sie die Modellunktion $h$. Die Funktion $h$ soll die gleichen Parameter wie $f$ erhalten und die Funktion $f$ intern aufrufen. Das Ergebnis von $f$ soll durch die Sigmoid-Aktivierungsfunktion transformiert werden.:** $$\hat y = h_{\theta}(x) = \sigma(Z) = \frac{1}{1+e^{-Z}}$$

In [12]:
def h(X, theta):
    """returns the sigmoid of the linear function.
    Arguments:
        X: Data
        theta: Parameters
    """
    
    Z = f(X, theta)

    y_hat = 1/(1+np.exp(-Z))
    
    return y_hat

In [13]:
# Test Cell
#----------

y_hat = h(X_c, theta_c)
#----------
# h

assert y_hat.shape == (m_c, 1)
assert np.isclose(y_hat[0], 0.96713), 'Expected 0.96713 but got %.5f' %y_hat[0]

del y_hat

**Aufgabe: Berechnen Sie nun die Kostenfunktion. Schreiben Sie eine Funktion $J$, die folgende Parameter erhält:**
1. Die Matrix $X \in \mathbb{R}^{m\times{}n}$, die die Datenpunkte des Trainigsdatensatzes enthält. .
2. Die Parameter $\theta$.
3. Die Label $y$ in der Größe des Datensatzes `(m, 1)`.

**$J$ berechnet die folgende Kostenfunktion:** $$J_{\theta}(x)=-\frac{1}{m} \sum\limits_{i = 1}^{m} [y^{(i)}\log(\hat y^{(i)}) + (1-y^{(i)})\log(1- \hat y^{(i)})]$$

In [221]:
def J(X,theta,y):
    """computes the Cross-entropy cost function
    Arguments:
        X: Data
        theta: Parameter
        y: True labels
    """

    m = len(X)
    y_hat = h(X, theta)
    result_sum = 0
    for i in range(1, m + 1):
      result_sum += ((y[i-1]) * (np.log((y_hat[i-1])))
      + ((1 - (y[i-1])) * np.log(1 - (y_hat[i-1]))))

    result = (-(1 / m)) * result_sum

    return result.squeeze()

In [222]:
# Test Cell
#----------

cost = J(X_c,theta_c,y_c)
#----------
# J

assert cost.shape == (), 'Use correctly sequenced matrix multiplication'
assert np.isclose(cost, 0.66739), 'Expected 0.66739but got %.5f' %cost

del cost

## Gradientenverfahren

**Schreibe für das Gradientenverfahren eine Funktion `grads`, die folgende Parameter erhält**
1. Die Matrix $X \in \mathbb{R}^{m\times{}n}$, die die Datenpunkte des Trainigsdatensatzes enthält. .
2. Die Parameter $\theta$.
3. Die Label $y$ in der Größe des Datensatzes `(m, 1)`.

**und den Gradienten $\partial\theta$ für die Parameter $\theta$ berechnet** 

Dabei ist $\partial\theta$ ein Vektor der Dimension `(n+1, 1)` mit den Gradienten der Parameter: $$ \partial \theta = \frac{1}{m}X^T(\hat y-y)$$

In [235]:
def grads(X,theta,y):
    """Berechnet die Gradienten der Kostenfunktion abhängig von dern Parametern.
    Arguments:
        X: Data
        theta: Parameter
        y: True labels
    """
    
    m = len(X)
    y_hat = h(X, theta)
    xt = np.matrix.transpose(X)

    print(xt)
    print(np.subtract(y_hat, y))

    dtheta = (1 / m) * np.matmul(xt, np.subtract(y_hat, y))
    
    return dtheta

In [236]:
# Test Cell
#----------

dt = grads(X_c,theta_c,y_c)
#----------
# grads

assert dt.shape == theta_c.shape
assert np.isclose(dt[1], 0.38273), 'Expected 0.38273 but got %.5f' %dt[1]

[[ 1.          1.        ]
 [ 1.76405235  1.86755799]
 [ 0.40015721 -0.97727788]
 [ 0.97873798  0.95008842]
 [ 2.2408932  -0.15135721]]
[[0.20610183]
 [0.21518991]]


**Aufgabe: Schreiben Sie nun eine Funktion, die die Modellparameter aufgrund der berechneten Gradienten aktualisiert.Die Funktion `update`erhält die Parameter $\theta$, die Gradienten $\partial \theta$ sowie die Lernrate $\alpha$ und berechnet:**

$$\theta = \theta - \alpha \cdot \partial \theta$$

In [ ]:
def update(theta, dtheta, alpha):
    """updates parameters using gradient decent updating rule."""
    
    # YOUR CODE HERE
    raise NotImplementedError()
    
    return theta

In [ ]:
# Test Cell
#----------

t = update(theta_c, dt, 0.1)
#----------
# update

assert t.shape == theta_c.shape
assert np.isclose(t[1], -0.141491), 'Expected -0.141491 but got %.5f' %t[1]
del t, dt

Nun können wir das iterative Gradientenverfahren programmieren.
**Aufgabe: Schreiben Sie eine Funktion `gradient_descent`, die folgende Parameter erhät:**
1. Die Matrix $X \in \mathbb{R}^{m\times{}n}$, die die Datenpunkte des Trainigsdatensatzes enthält. .
2. Die Parameter $\theta$.
3. Die Label $y$ in der Größe des Datensatzes `(m, 1)`.
4. Die Lernrate $\alpha$.
5. Die Anzahl der Iterationen.
**Die Funktion soll die Trainierten Modellparameter $\theta$ zurückgeben.**

*Hinweis*: Berechnen Sie Kosten mit der Funktion `J` und hängen Sie diese Kosten nach jedem Berechnungsschritt and die Liste `cost` an $\rightarrow$ Berechnen Sie die Gradienten mit der Funktion `grads` $\rightarrow$ Verwenden Sie diese Gradienten um die Parameter mit der Funktion `update`

In [ ]:
def gradient_decent(X, theta, y, alpha=0.1, iterations=100):
    """performs gradient decent optimization.
    Arguments:
        X: Data
        theta: Parameter
        y: True labels
        alpha(default=0.1): Learning rate
        iterations(default=100): number of updating iterations
    """
    
    costs = []
    
    for i in range(iterations):
        # YOUR CODE HERE
        raise NotImplementedError()
        
    return theta, costs

In [ ]:
# Test Cell
#----------

t, costs = gradient_decent(X_c, theta_c, y_c)
#----------
# gradient_decent

assert len(costs) == 100, 'Make sure to calculate and append the cost in every iteration.'
assert np.isclose(t[4], 1.05769), 'Expected 1.05186 but got %.5f' %t[4]
plt.plot([i for i in range(len(costs))],costs)
del t, costs

## Anwendung der Funktionen auf einen realistischen Datensatz

Wir habe nun alle Funktionen um unser logistisches Regressionsmodell für einen *echten* Datensatz zu einzusetzen.
Wir verwenden hier den Brustkrebs-Datensatz aus Sklearn:

In [ ]:
scaler = MinMaxScaler()
data = load_breast_cancer()

X_train, X_test, y_train, y_test = train_test_split(data.data,data.target,test_size=0.3)

# preprocessing
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
y_train = np.expand_dims(y_train, 1)
y_test = np.expand_dims(y_test, 1)

X_train = np.c_[np.ones(X_train.shape[0]).T,X_train]
X_test = np.c_[np.ones(X_test.shape[0]).T,X_test]

In [ ]:
# initializing parameters
theta = np.zeros((len(X_train[0]), 1))

#training the model
theta, costs = gradient_decent(X_train, theta, y_train.reshape(-1, 1), alpha = 2)

In [ ]:
plt.plot(range(1,len(costs)),costs[1:], "x-")
plt.show()

In [ ]:
# measuring performance
y_pred = (h(X_test,theta) >= 0.5)*1
acc = 100-np.sum(np.abs(y_pred-y_test))*100/len(y_test)

print("Die classifcation accuracy ist: ",acc)